In [1]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame 

import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
%matplotlib inline
from matplotlib.pyplot import figure
import matplotlib as mpl
import matplotlib.font_manager as fm
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'Malgun Gothic' 
plt.rcParams["font.size"] = 10
%matplotlib inline 


from datetime import datetime, timedelta 
from statsmodels.tsa.arima_model import ARIMA 

from sklearn.preprocessing import StandardScaler
import os

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Weather 추가변수
- 나중에 모델 돌릴때마다 df에 추가하는 것

In [3]:
# hourly WEATHER data
w_hr = pd.read_csv("data/kaggle/weather/hourly.csv",encoding='euc-kr', sep=',')
w_hr2 = w_hr.loc[:,['Time', 'Temp','Humidity', 'WindSpeed']]
w_hr2['Humidity']=w_hr2.loc[:,'Humidity'].fillna(0)
w_hr2.head(2)

,Time,Temp,Humidity,WindSpeed
0,2016.7.20 0:00,25.7,0.0,1.3
1,2016.7.20 1:00,25.6,0.0,1.4


In [4]:
w_hr2['Time'] = pd.to_datetime(w_hr2.Time)
w_hr2 = w_hr2.set_index('Time')
w_hr2.head(2)

,Temp,Humidity,WindSpeed
Time,,,
2016-07-20 00:00:00,25.7,0.0,1.3
2016-07-20 01:00:00,25.6,0.0,1.4


## 파생 변수

In [6]:
test = pd.read_csv("data/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')

In [7]:
test.index = pd.to_datetime(test.index)

In [38]:
test

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431000,0.276000,0.228000,0.432000,0.232000,0.348000,0.259000,0.194000,0.086000,...,0.114000,0.087000,0.283000,0.342142,1.148489,0.094000,0.351000,1.109,0.225,0.296670
2017-08-29 11:00:00,1.458903,0.398000,0.293000,0.358000,0.350000,0.232000,0.253000,0.260000,0.168000,0.084000,...,0.163000,0.086000,0.288000,0.342010,1.150483,0.196000,0.343000,1.109,0.225,0.296670
2017-08-29 12:00:00,1.462694,0.427000,0.383000,0.462000,0.410000,0.228000,0.243000,0.429000,0.146000,0.054000,...,0.192000,0.181000,0.323000,0.341950,1.151945,0.111000,0.322000,1.109,0.225,0.296670
2017-08-29 13:00:00,1.465513,0.727000,0.351000,0.608000,0.491000,0.234000,0.243000,0.458000,0.184000,0.059000,...,0.121000,0.141000,0.339000,0.341925,1.153084,0.110000,0.203000,1.109,0.225,0.296670
2017-08-29 14:00:00,1.467609,1.208000,0.336000,0.454000,0.404000,0.242000,0.228000,0.333000,0.146000,0.055000,...,0.214000,0.096000,0.241000,0.341920,1.154028,0.106000,0.133000,1.109,0.225,0.296671
2017-08-29 15:00:00,1.469168,0.819000,0.308000,0.342000,0.448000,0.213000,0.235000,0.255000,0.149000,0.076000,...,0.125000,0.096000,0.277000,0.341924,1.154852,0.077000,0.136000,1.109,0.225,0.296672
2017-08-29 16:00:00,1.470327,0.438000,0.250000,0.269000,0.414000,0.248000,0.235000,0.355000,0.163000,0.118000,...,0.177000,0.092000,0.241000,0.341933,1.155604,0.096000,0.091000,1.109,0.225,0.296673
2017-08-29 17:00:00,1.471188,0.417000,0.239000,0.260000,0.731000,0.215000,0.275000,0.496000,0.148000,0.203000,...,0.284000,0.091000,0.264000,0.341944,1.156312,0.093000,0.168000,1.109,0.225,0.296673
2017-08-29 18:00:00,1.471829,0.457000,0.328000,0.272000,0.537000,0.247000,0.298000,0.471000,0.140000,0.343000,...,0.146000,0.089000,0.425000,0.341957,1.156994,0.093000,0.116000,1.109,0.225,0.296674


In [8]:
def dateTimeVar(df):
    df1 = pd.DataFrame(df.index)
    df1.index = pd.to_datetime(df1.iloc[:,0])
    df1 = df1.drop(columns=df1.columns)
    # datetime을 사용한 파생변수
    # 연도, 월, 일, 요일, 시간
    df1['Mth']= df1.index.month
    df1['Date']=df1.index.day
    df1['Day']=df1.index.weekday
    df1['hour']=df1.index.hour
    
    # Season 1: 봄 ~ 4: 겨울 
    df1['season'] = df1.Mth.apply(lambda x: 4 if x in [12,1,2] else 
                1 if x in [3,4,5] else 2 if x in [6,7,8]
               else 3)
    # dayClass 1: 월, 2: 화~금, 3: 토, 일 
    # 참조 from 실시간 수요예측 기법 개발 및 온라인 수요예측 방안 연구_KPX 한국전력거래소)
    df1['dayClass'] = df1.Day.apply(lambda x: 1 if x ==0 else 
                2 if x in [1,2,3,4] else 3)
    return df1

In [9]:
TE_time_vars = dateTimeVar(test)

In [10]:
TE_time_vars.head(1)

,Mth,Date,Day,hour,season,dayClass
Time,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2


## Make DUMMIES

In [11]:
test.head(2)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667
2017-08-29 11:00:00,1.458903,0.398,0.293,0.358,0.350,0.232,0.253,0.260,0.168,0.084,...,0.163,0.086,0.288,0.342010,1.150483,0.196,0.343,1.109,0.225,0.29667


In [12]:
TE_time_vars.head(2)

,Mth,Date,Day,hour,season,dayClass
Time,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2
2017-08-29 11:00:00,8,29,1,11,2,2


In [13]:
def dummies(df, dum_lst):
    df1 = df.copy()
    # dum_lst에 들어있는 각각의 변수마다 
    # tmp = get_dummies, 해당 original 변수는 drop
    # tmp와 변수가 drop된 df를 join 후 리턴
    for v in dum_lst:
        tmp = pd.get_dummies(df1[v], prefix=v)
        df1 = df1.drop(columns=v).join(tmp)
    return df1

In [14]:
TE_timeV_dum = dummies(TE_time_vars, ['season', 'dayClass'])

In [65]:
TE_timeV_dum = TE_time_vars1.copy()

In [66]:
TE_timeV_dum_wthr= pd.concat([TE_timeV_dum, w_hr2], axis=1,join='inner')

In [67]:
TE_timeV_dum_wthr.head(2)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0,19.3,0.0,5.0
2017-08-29 11:00:00,8,29,1,11,0,1,0,0,0,1,0,19.7,0.0,4.7


# Modeling

* test : TEST셋의 순수한 meterIDs Time series (NA값 채워짐) 
* TE_timeV_dum: TEST셋의 시간파생변수 + dummy variables 

In [17]:
test.head(1)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667


In [18]:
TE_timeV_dum_wthr.head(1)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
Time,,,,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0,19.3,0.0,5.0


**### 칼럼 10~50개 돌리기

In [19]:
##################################################
#### 나눠서 돌림

test2 = test.iloc[:,:2]
test2.head(1)

,X26,X303
Time,,
2017-08-29 10:00:00,1.453803,0.431


In [20]:
submission_hourly = pd.date_range(start = pd.datetime(2018, 7, 1, 0),
                             end = pd.datetime(2018, 7, 1, 23),
                             freq = "H")
tmp_test = pd.DataFrame(index = submission_hourly)

In [68]:
X_test_H = pd.concat([X_test_H1, w_hr2], axis=1,join='inner')

In [69]:
X_test_H.head(3)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
2018-07-01 00:00:00,7,1,6,0,0,1,0,0,0,0,1,23.7,0.0,1.7
2018-07-01 01:00:00,7,1,6,1,0,1,0,0,0,0,1,23.7,0.0,2.0
2018-07-01 02:00:00,7,1,6,2,0,1,0,0,0,0,1,23.8,0.0,2.0


In [58]:
Mer = pd.concat([TE_time_vars, X_test_H])
Mer


,Mth,Date,Day,hour,season,dayClass
2017-08-29 10:00:00,8,29,1,10,2,2
2017-08-29 11:00:00,8,29,1,11,2,2
2017-08-29 12:00:00,8,29,1,12,2,2
2017-08-29 13:00:00,8,29,1,13,2,2
2017-08-29 14:00:00,8,29,1,14,2,2
2017-08-29 15:00:00,8,29,1,15,2,2
2017-08-29 16:00:00,8,29,1,16,2,2
2017-08-29 17:00:00,8,29,1,17,2,2
2017-08-29 18:00:00,8,29,1,18,2,2
2017-08-29 19:00:00,8,29,1,19,2,2


In [59]:
Mer1 = dummies(Mer, ['season', 'dayClass'])
Mer1

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0
2017-08-29 11:00:00,8,29,1,11,0,1,0,0,0,1,0
2017-08-29 12:00:00,8,29,1,12,0,1,0,0,0,1,0
2017-08-29 13:00:00,8,29,1,13,0,1,0,0,0,1,0
2017-08-29 14:00:00,8,29,1,14,0,1,0,0,0,1,0
2017-08-29 15:00:00,8,29,1,15,0,1,0,0,0,1,0
2017-08-29 16:00:00,8,29,1,16,0,1,0,0,0,1,0
2017-08-29 17:00:00,8,29,1,17,0,1,0,0,0,1,0
2017-08-29 18:00:00,8,29,1,18,0,1,0,0,0,1,0
2017-08-29 19:00:00,8,29,1,19,0,1,0,0,0,1,0


In [63]:
TE_time_vars1 = Mer1.iloc[:7334].copy()
TE_time_vars1

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0
2017-08-29 11:00:00,8,29,1,11,0,1,0,0,0,1,0
2017-08-29 12:00:00,8,29,1,12,0,1,0,0,0,1,0
2017-08-29 13:00:00,8,29,1,13,0,1,0,0,0,1,0
2017-08-29 14:00:00,8,29,1,14,0,1,0,0,0,1,0
2017-08-29 15:00:00,8,29,1,15,0,1,0,0,0,1,0
2017-08-29 16:00:00,8,29,1,16,0,1,0,0,0,1,0
2017-08-29 17:00:00,8,29,1,17,0,1,0,0,0,1,0
2017-08-29 18:00:00,8,29,1,18,0,1,0,0,0,1,0
2017-08-29 19:00:00,8,29,1,19,0,1,0,0,0,1,0


In [64]:
X_test_H1 = Mer1.iloc[7334:7358].copy()
X_test_H1

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3
2018-07-01 00:00:00,7,1,6,0,0,1,0,0,0,0,1
2018-07-01 01:00:00,7,1,6,1,0,1,0,0,0,0,1
2018-07-01 02:00:00,7,1,6,2,0,1,0,0,0,0,1
2018-07-01 03:00:00,7,1,6,3,0,1,0,0,0,0,1
2018-07-01 04:00:00,7,1,6,4,0,1,0,0,0,0,1
2018-07-01 05:00:00,7,1,6,5,0,1,0,0,0,0,1
2018-07-01 06:00:00,7,1,6,6,0,1,0,0,0,0,1
2018-07-01 07:00:00,7,1,6,7,0,1,0,0,0,0,1
2018-07-01 08:00:00,7,1,6,8,0,1,0,0,0,0,1
2018-07-01 09:00:00,7,1,6,9,0,1,0,0,0,0,1


In [70]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

lg_param = {"eta" : [0.01, 0.05, 0.1, 0.2],
           "Min_child_weight" : [0.1, 0.5, 1],
           "Max_depth" : [3, 5, 6, 10]}

def gsLGBR_Hrly_FitPred (model, params, mID, X_train, y_train, X_test):
    grid_gb = GridSearchCV(model, params, cv=4, verbose=5)
    grid_gb.fit(X_train, y_train)
    fcst = grid_gb.predict(X_test)
    a = pd.DataFrame([mID], columns=['meter_id'])
    for i in range(24):
        a['X2018_7_1_'+str(i+1)+'h']=[fcst[i]]
    return a
   
def test50_Hrly_GS_LGBReg(train, X_test_H):
    hrly_50 = pd.DataFrame()
    for mID in train.columns:
        data = pd.concat([train.loc[:,[mID]], TE_timeV_dum_wthr], axis=1)
        data = data.rename(columns={mID:'y'})
        for i in [24, 168]:
            data['lag_{}'.format(i)] = data.y.shift(i)
        y_train = data.dropna().y
        X_train = data.dropna().drop(['y'], axis=1)
        
        X_test_H['lag_24'] = y_train.tail(24).values
        X_test_H['lag_168'] = data.loc[:,'lag_168'][7166:7190].values
        
        model = xgb.XGBRegressor(n_estimators=1000)
        tmp = gsLGBR_Hrly_FitPred(model, lg_param,
                        mID, X_train, y_train, X_test_H)
        hrly_50 = pd.concat([hrly_50, tmp], axis=0)
    return hrly_50

In [71]:
test1 = test2.loc[:, ['X303']]

In [72]:
pred_2 = test50_Hrly_GS_LGBReg(test1, X_test_H)

Fitting 4 folds for each of 48 candidates, totalling 192 fits
[CV] Max_depth=3, Min_child_weight=0.1, eta=0.01 .....................
[11:07:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\envs\[tf]\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  Max_depth=3, Min_child_weight=0.1, eta=0.01, score=0.253, total=   3.8s
[CV] Max_depth=3, Min_child_weight=0.1, eta=0.01 .....................
[11:07:13] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.7s remaining:    0.0s
C:\ProgramData\Anaconda3\envs\[tf]\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  Max_depth=3, Min_child_weight=0.1, eta=0.01, score=-0.019, total=   3.8s
[CV] Max_depth=3, Min_child_weight=0.1, eta=0.01 .....................
[11:07:17] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

C:\ProgramData\Anaconda3\envs\[tf]\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  Max_depth=10, Min_child_weight=1, eta=0.2, score=0.125, total=   3.9s
[CV] Max_depth=10, Min_child_weight=1, eta=0.2 .......................
[11:19:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\ProgramData\Anaconda3\envs\[tf]\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  Max_depth=10, Min_child_weight=1, eta=0.2, score=-0.099, total=   3.8s
[11:19:18] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Done 192 out of 192 | elapsed: 12.1min finished
C:\ProgramData\Anaconda3\envs\[tf]\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\envs\[tf]\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [73]:
pred_2

,meter_id,X2018_7_1_1h,X2018_7_1_2h,X2018_7_1_3h,X2018_7_1_4h,X2018_7_1_5h,X2018_7_1_6h,X2018_7_1_7h,X2018_7_1_8h,X2018_7_1_9h,...,X2018_7_1_15h,X2018_7_1_16h,X2018_7_1_17h,X2018_7_1_18h,X2018_7_1_19h,X2018_7_1_20h,X2018_7_1_21h,X2018_7_1_22h,X2018_7_1_23h,X2018_7_1_24h
0,X303,0.728295,0.682456,0.976058,0.832435,0.620824,0.572198,0.483261,0.491112,0.539415,...,1.53427,1.540973,1.558024,1.466103,1.478747,0.963869,0.906838,0.864462,0.745418,0.660964


In [69]:
import os
pred_2.to_csv(r'first2_Hrly.csv')

from IPython.display import FileLink
FileLink(r'first2_Hrly.csv')

/kaggle/working/first2_Hrly.csv